Fetch Rewards Coding Exercise - Part Two and Three

Writing SQL queries to address the following business problems and evaluating the data for quality using Python and SQLite:

1. When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
2. When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [50]:
# Import Essential Libraries

import sqlite3
import json
import pandas as pd
from sqlite3 import Error
import matplotlib.pyplot as plt
%matplotlib inline

In [51]:
def load_json_file(filename):
    """Reads a JSON file and handles JSON decoding errors."""
    with open(filename, 'r') as file:
        try:
            data = json.load(file)  # Attempt to load as a single JSON object
        except json.JSONDecodeError:
            file.seek(0)  # Reset file pointer
            data = [json.loads(line) for line in file]  # Handle JSONL (newline-delimited JSON)
    return data

# Load JSON files
users_data = load_json_file('users.json')
brands_data = load_json_file('brands.json')
receipts_data = load_json_file('receipts.json')

# Convert to Pandas DataFrames
users_df = pd.DataFrame(pd.json_normalize(users_data, sep='_'))
brands_df = pd.DataFrame(pd.json_normalize(brands_data, sep='_'))
receipts_df = pd.DataFrame(pd.json_normalize(receipts_data, sep='_'))


Perform EDA to understand the datasets

The three datasets that are provided include both numeric and string (oject in Pandas) columns.

In [52]:
brands_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   barcode       1167 non-null   object
 1   category      1012 non-null   object
 2   categoryCode  517 non-null    object
 3   name          1167 non-null   object
 4   topBrand      555 non-null    object
 5   _id_$oid      1167 non-null   object
 6   cpg_$id_$oid  1167 non-null   object
 7   cpg_$ref      1167 non-null   object
 8   brandCode     933 non-null    object
dtypes: object(9)
memory usage: 82.2+ KB


In [53]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   active             495 non-null    bool   
 1   role               495 non-null    object 
 2   signUpSource       447 non-null    object 
 3   state              439 non-null    object 
 4   _id_$oid           495 non-null    object 
 5   createdDate_$date  495 non-null    int64  
 6   lastLogin_$date    433 non-null    float64
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 23.8+ KB


In [54]:
receipts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bonusPointsEarned        544 non-null    float64
 1   bonusPointsEarnedReason  544 non-null    object 
 2   pointsEarned             609 non-null    object 
 3   purchasedItemCount       635 non-null    float64
 4   rewardsReceiptItemList   679 non-null    object 
 5   rewardsReceiptStatus     1119 non-null   object 
 6   totalSpent               684 non-null    object 
 7   userId                   1119 non-null   object 
 8   _id_$oid                 1119 non-null   object 
 9   createDate_$date         1119 non-null   int64  
 10  dateScanned_$date        1119 non-null   int64  
 11  finishedDate_$date       568 non-null    float64
 12  modifyDate_$date         1119 non-null   int64  
 13  pointsAwardedDate_$date  537 non-null    float64
 14  purchaseDate_$date      

Although there aren't many rows with null values in the Users Dataset, there are many rows with null values in the dataset brands and receipts. If the columns with null values aren't necessary for the analysis, they can be left alone; if not, they must be filled in with a substitute value or, if the dataset is large enough, should be removed.

In [55]:
print('Receipts')
print('--------------------------')
print(receipts_df.isnull().sum())
print('--------------------------')
print('Users')
print('--------------------------')
print(users_df.isnull().sum())
print('--------------------------')
print('Brands')
print('--------------------------')
print(brands_df.isnull().sum())

Receipts
--------------------------
bonusPointsEarned          575
bonusPointsEarnedReason    575
pointsEarned               510
purchasedItemCount         484
rewardsReceiptItemList     440
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
_id_$oid                     0
createDate_$date             0
dateScanned_$date            0
finishedDate_$date         551
modifyDate_$date             0
pointsAwardedDate_$date    582
purchaseDate_$date         448
dtype: int64
--------------------------
Users
--------------------------
active                0
role                  0
signUpSource         48
state                56
_id_$oid              0
createdDate_$date     0
lastLogin_$date      62
dtype: int64
--------------------------
Brands
--------------------------
barcode           0
category        155
categoryCode    650
name              0
topBrand        612
_id_$oid          0
cpg_$id_$oid      0
cpg_$ref          0
brandCode       234
dtyp

The dataset contains 495 users, 283 of them are duplicates.

In [56]:
print(users_df.duplicated().sum()) # duplicates in users table
print(brands_df.duplicated().sum())
print(receipts_df.duplicated(subset = ['_id_$oid']).sum())

283
0
0


In [57]:
#create a dict to store the itemlist against its receiptId
receipt_items = {}
for index , receipt in receipts_df.iterrows():
    #check for nan
    if receipt['rewardsReceiptItemList'] != receipt['rewardsReceiptItemList']:
        pass
    else:
        receipt_items[receipt['_id_$oid']] = receipt['rewardsReceiptItemList']

In [58]:
import pandas as pd

# Initialize an empty list to store row dictionaries
flattened_data = []

# Loop through each receipt and its reward items
for receipt_id, reward_items in receipt_items.items():
    for item in reward_items:
        # Create a copy of the item's data and add the receiptId
        row = item.copy()
        row['receiptId'] = receipt_id
        flattened_data.append(row)  # Append to the list

# Convert list of dictionaries into a DataFrame
itemlist_df = pd.DataFrame(flattened_data)

# Display first few rows
itemlist_df.head()
#print(itemlist_df.to_string(index=False))  # Prints full rows without cutting off data


,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
itemlist_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6941 entries, 0 to 6940
Data columns (total 35 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   barcode                             3090 non-null   object 
 1   description                         6560 non-null   object 
 2   finalPrice                          6767 non-null   object 
 3   itemPrice                           6767 non-null   object 
 4   needsFetchReview                    813 non-null    object 
 5   partnerItemId                       6941 non-null   object 
 6   preventTargetGapPoints              358 non-null    object 
 7   quantityPurchased                   6767 non-null   float64
 8   userFlaggedBarcode                  337 non-null    object 
 9   userFlaggedNewItem                  323 non-null    object 
 10  userFlaggedPrice                    299 non-null    object 
 11  userFlaggedQuantity                 299 non

In [60]:

# Display all brandCode counts

display(itemlist_df['brandCode'].value_counts())
pd.reset_option('display.max_rows')  # Reset to default after displaying

,count
brandCode,
HY-VEE,291
BEN AND JERRYS,180
PEPSI,93
KROGER,89
KLEENEX,88
...,...
PACIFIC FOODS,1
EGGO,1
GRIMMWAY FARMS,1


In [61]:
#count of receipts for each receipt status
receipts_df['rewardsReceiptStatus'].value_counts() # no receipts for accepted status

,count
rewardsReceiptStatus,
FINISHED,518
SUBMITTED,434
REJECTED,71
PENDING,50
FLAGGED,46


Build tables in a SQLite database using the data model as a guide.

In [62]:
import sqlite3

In [63]:
# Function to connect to the SQLite database
def connect_database(db):
    '''connect to the database mentioned in arguments'''
    conn = None
    try:
        conn = sqlite3.connect(db)
    except Error as e:
        print(e)
    return conn

In [64]:
# Function to create a table in the database
def create_table(conn, create_table_sql):
    """create a table from the create_table_sql statement"""
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [65]:
# Function to insert records into the database
def insert_records(conn, insert_sql, data):
    '''inserts data into tables mentioned in insert_sql statement'''
    try:
        cur = conn.cursor()
        for row in data:
            cur.execute(insert_sql, row)  # Inserting each row of data into the table
        conn.commit()  # Committing the changes
    except Error as e:
        print(e)
    return cur.lastrowid  # Return the ID of the last inserted row

In [66]:
# Function to select and display results from a query
def select_query(conn, select_sql):
    '''executes a select query'''
    try:
        cur = conn.cursor()
        cur.execute(select_sql)
        columns = [col[0] for col in cur.description]  # Get column names
        print(columns)  # Print column names
        rows = cur.fetchall()  # Fetch all rows from the query
        for row in rows:
            print(row)  # Print each row
    except Error as e:
        print(e)

In [67]:
import sqlite3
import os

# Define the folder path where you want to save the database (C: drive in this case)
folder_path = r'C:\connection_data'

# Create the folder if it doesn't exist
os.makedirs(folder_path, exist_ok=True)

# Define the path to your database file
db_path = os.path.join(folder_path, 'rewards_db.db')  # Database name will be 'rewards_db.db'

# Function to connect to the database
def connect_database(db):
    '''Connect to the database'''
    conn = None
    try:
        conn = sqlite3.connect(db)  # Connect to SQLite database
        print(f"Successfully connected to the database at {db}")
    except sqlite3.Error as e:
        print(f"Error connecting to the database: {e}")
    return conn

# Connect to the database
conn = connect_database(db_path)

# Close the connection after use
if conn is not None:
    conn.close()
else:
    print("Failed to connect to the database!")


Successfully connected to the database at C:\connection_data/rewards_db.db


In [71]:
import sqlite3

# Connect to your SQLite database
conn = sqlite3.connect('path_to_your_database.db')
cursor = conn.cursor()

# SQL commands to create tables
create_receipts_table_sql = '''CREATE TABLE Receipts (
    receiptId TEXT PRIMARY KEY,
    bonusPointsEarnedReason TEXT,
    rewardsReceiptStatus TEXT
);'''

create_rewardpoints_table_sql = '''CREATE TABLE RewardsPoints (
    receiptId TEXT,
    userId TEXT,
    createdDateId INTEGER,
    dateScannedId INTEGER,
    finishedDateId INTEGER,
    modifyDateId INTEGER,
    pointsAwardedDateId INTEGER,
    purchaseDateId INTEGER,
    bonusPointsEarned REAL,
    pointsEarned REAL,
    purchaseItemCount INTEGER,
    totalSpent REAL,
    FOREIGN KEY (receiptId) REFERENCES Receipts (receiptId)
);'''

# Execute SQL commands to create tables
cursor.execute(create_receipts_table_sql)
cursor.execute(create_rewardpoints_table_sql)

# Commit and close the connection
conn.commit()
conn.close()


In [72]:
import sqlite3
import os

# Define the database folder path
folder_path = r'C:\connection_data'
os.makedirs(folder_path, exist_ok=True)  # Ensure the folder exists

# Database file path
db_path = os.path.join(folder_path, 'rewards_db.db')

# Function to connect to the database
def connect_database(db):
    '''Connect to SQLite database'''
    conn = None
    try:
        conn = sqlite3.connect(db)  # Connect to database
        print(f"Successfully connected to the database at {db}")
    except sqlite3.Error as e:
        print(f"Error connecting to database: {e}")
    return conn

# Function to create tables
def create_tables(conn):
    try:
        cursor = conn.cursor()

        # Create Receipts table
        create_receipts_table_sql = '''CREATE TABLE IF NOT EXISTS Receipts(
                                        receiptId TEXT PRIMARY KEY,
                                        bonusPointsEarnedReason TEXT,
                                        rewardsReceiptStatus TEXT
                                        );'''

        # Create RewardsPoints table
        create_rewardpoints_table_sql = '''CREATE TABLE IF NOT EXISTS RewardsPoints(
                            receiptId TEXT,
                            userId TEXT,
                            createdDateId INTEGER,
                            dateScannedId INTEGER,
                            finishedDateId INTEGER,
                            modifyDateId INTEGER,
                            pointsAwardedDateId INTEGER,
                            purchaseDateId INTEGER,
                            bonusPointsEarned REAL,
                            pointsEarned REAL,
                            purchaseItemCount INTEGER,
                            totalSpent REAL,
                            FOREIGN KEY (receiptId) REFERENCES Receipts (receiptId)
                            );'''

        # Execute table creation
        cursor.execute(create_receipts_table_sql)
        cursor.execute(create_rewardpoints_table_sql)

        conn.commit()  # Save changes
        print("Tables 'Receipts' and 'RewardsPoints' created successfully!")
    except sqlite3.Error as e:
        print(f"Error creating tables: {e}")

# Connect to the database
conn = connect_database(db_path)

# Create tables if connection was successful
if conn is not None:
    create_tables(conn)
    conn.close()  # Close connection
else:
    print("Failed to connect to the database!")


Successfully connected to the database at C:\connection_data/rewards_db.db
Tables 'Receipts' and 'RewardsPoints' created successfully!


In [73]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect(r'C:\connection_data\rewards_db.db')

# Function to create tables
def create_tables(conn):
    create_receipts_table_sql = '''CREATE TABLE IF NOT EXISTS Receipts(
                                    receiptId TEXT PRIMARY KEY,
                                    bonusPointsEarnedReason TEXT,
                                    rewardsReceiptStatus TEXT
                                    );'''

    create_rewardpoints_table_sql = '''CREATE TABLE IF NOT EXISTS RewardsPoints(
                                        receiptId TEXT,
                                        userId TEXT,
                                        bonusPointsEarned REAL,
                                        pointsEarned REAL,
                                        purchaseItemCount INTEGER,
                                        totalSpent REAL,
                                        FOREIGN KEY (receiptId) REFERENCES Receipts (receiptId)
                                        );'''

    try:
        cur = conn.cursor()
        cur.execute(create_receipts_table_sql)  # Create Receipts table
        cur.execute(create_rewardpoints_table_sql)  # Create RewardsPoints table
        conn.commit()
        print("Tables created successfully.")
    except sqlite3.Error as e:
        print(f"SQLite error: {e}")
    finally:
        cur.close()

# Call the function to create tables
create_tables(conn)

# Verify tables exist
cur = conn.cursor()
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()
print("Existing Tables in Database:", tables)
cur.close()

# Now, try inserting records
insert_into_receipts = '''INSERT INTO Receipts(receiptId, bonusPointsEarnedReason, rewardsReceiptStatus)
                          VALUES(?, ?, ?);'''

insert_into_rewardpoints = '''INSERT INTO RewardsPoints(receiptId, userId, bonusPointsEarned, pointsEarned, purchaseItemCount, totalSpent)
                              VALUES(?, ?, ?, ?, ?, ?);'''

# Ensure receipts_df has required columns
receipts_data = zip(receipts_df['_id_$oid'], receipts_df['bonusPointsEarnedReason'], receipts_df['rewardsReceiptStatus'])
rewardpoints_data = zip(receipts_df['_id_$oid'], receipts_df['userId'], receipts_df['bonusPointsEarned'], receipts_df['pointsEarned'], receipts_df['purchasedItemCount'], receipts_df['totalSpent'])

# Function to insert records
def insert_records(conn, query, data):
    try:
        cur = conn.cursor()
        cur.executemany(query, data)
        conn.commit()
        print("Data inserted successfully.")
    except sqlite3.Error as e:
        print(f"SQLite error: {e}")
    finally:
        cur.close()

# Insert data
if conn is not None:
    try:
        insert_records(conn, insert_into_receipts, receipts_data)
        insert_records(conn, insert_into_rewardpoints, rewardpoints_data)
    except Exception as e:
        print(f"Error while inserting records: {e}")
    finally:
        conn.close()
        print("Database connection closed.")
else:
    print("Cannot Insert Records. Error in connecting to the database!")


Tables created successfully.
Existing Tables in Database: [('Receipts',), ('RewardsPoints',)]
SQLite error: UNIQUE constraint failed: Receipts.receiptId
Data inserted successfully.
Database connection closed.


FINAL OUTPUT

(Note: Since the dataset did not have any receipts for Accepted status, I included receipts of Finished status and it looks like receipts with status finished have the greater average spend as well as the total number of items purchased.)

In [74]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect(r'C:\connection_data\rewards_db.db')

# Function to execute SELECT queries
def select_query(conn, query):
    try:
        cur = conn.cursor()
        cur.execute(query)
        rows = cur.fetchall()
        for row in rows:
            print(row)
        cur.close()
    except sqlite3.Error as e:
        print(f"SQLite error: {e}")

# SQL queries

# Average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’
avg_spent = '''SELECT r.rewardsReceiptStatus, AVG(rp.totalSpent) AS avgTotalSpent
               FROM RewardsPoints AS rp
               JOIN Receipts AS r ON rp.receiptId = r.receiptId
               WHERE r.rewardsReceiptStatus IN ('ACCEPTED', 'REJECTED', 'FINISHED')
               GROUP BY r.rewardsReceiptStatus
               ORDER BY avgTotalSpent;'''

# Total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’
total_num_items = '''SELECT r.rewardsReceiptStatus, SUM(rp.purchaseItemCount) AS totalNumItems
                     FROM RewardsPoints AS rp
                     JOIN Receipts AS r ON rp.receiptId = r.receiptId
                     WHERE r.rewardsReceiptStatus IN ('ACCEPTED', 'REJECTED', 'FINISHED')
                     GROUP BY r.rewardsReceiptStatus
                     ORDER BY totalNumItems;'''

# Run queries if connection is valid
if conn is not None:
    print("Executing avg_spent query:")
    select_query(conn, avg_spent)

    print('--------------------')

    print("Executing total_num_items query:")
    select_query(conn, total_num_items)

    # Close connection after queries
    conn.close()
else:
    print('Cannot execute the query. Error in connecting to the database!')


Executing avg_spent query:
('REJECTED', 23.326056338028188)
('FINISHED', 80.85430501930492)
--------------------
Executing total_num_items query:
('REJECTED', 346)
('FINISHED', 16368)


Since the dataset did not have any receipts for Accepted status, I included receipts of Finished status and it looks like receipts with status finished have the greater average spend as well as the total number of items purchased.

In [75]:
# Ensure data is in your DataFrame
print(receipts_df.head())

# Check database connection
if conn is not None:
    try:
        # Insert data into Receipts table
        insert_records(conn, insert_into_receipts, receipts_data)
        # Insert data into RewardsPoints table
        insert_records(conn, insert_into_rewardpoints, rewardpoints_data)
    except Exception as e:
        print(f"Error while inserting records: {e}")
else:
    print('Cannot insert records. Error in connecting to the database!')


   bonusPointsEarned                            bonusPointsEarnedReason  \
0              500.0  Receipt number 2 completed, bonus point schedu...   
1              150.0  Receipt number 5 completed, bonus point schedu...   
2                5.0                         All-receipts receipt bonus   
3                5.0                         All-receipts receipt bonus   
4                5.0                         All-receipts receipt bonus   

  pointsEarned  purchasedItemCount  \
0        500.0                 5.0   
1        150.0                 2.0   
2            5                 1.0   
3          5.0                 4.0   
4          5.0                 2.0   

                              rewardsReceiptItemList rewardsReceiptStatus  \
0  [{'barcode': '4011', 'description': 'ITEM NOT ...             FINISHED   
1  [{'barcode': '4011', 'description': 'ITEM NOT ...             FINISHED   
2  [{'needsFetchReview': False, 'partnerItemId': ...             REJECTED   
3  [{'barcod